In [1]:
def get_laplacian_scored(X,k=5,n=5):
    ''' Gets the top k parameters as per locality preservation adherence
        k: top k locality/nn graph preserving features
        n: nearest neighbor graph hyperparameter
        Returns: indices of top k features/columns of X
        
        Requirements:
        1.
        import numpy as np
        import scipy as sp
        from sklearn.neighbors import NearestNeighbors
        2.
        X is numeric and does not have Nulls or NaNs
        
        Demonstration:
        from sklearn.datasets import load_iris
        data = load_iris().data
        features = load_iris().feature_names
        [features[idx] for idx in get_laplacian_scored(data,k=2,n=2)]
    '''
    if  type(X) != np.ndarray:
        X = np.asarray(X)
    nbrs = NearestNeighbors(n_neighbors=n, algorithm='ball_tree').fit(X)
    G = nbrs.kneighbors()[1]
    S = []
    for i in range(len(G)):
        s = []
        for j in range(len(G)):
            if j in G[i,:]:
                s.append(np.exp(-1*sp.spatial.distance.euclidean(u=X[i],v=X[j])))
            else:
                s.append(0)
        S.append(s)
    S = np.array(S)
    d1 = np.ones(X.shape[0])
    factor_space = np.linalg.multi_dot([S,d1.transpose()])
    D = np.multiply(np.identity(X.shape[0]),factor_space)
    L = D - S
    f_n = []
    for j in range(X.shape[1]):
        f = X[:,j]
        f_n.append(f - np.multiply(np.linalg.multi_dot([np.linalg.multi_dot([f.transpose(),D]),d1])/np.linalg.multi_dot([np.linalg.multi_dot([d1.transpose(),D]),d1]),d1))
    f_n = np.array(f_n)
    lr = []
    for j in range(f_n.shape[0]):
        lr.append(np.linalg.multi_dot([np.linalg.multi_dot([f_n[j].transpose(),L]),f_n[j]])/np.linalg.multi_dot([np.linalg.multi_dot([f_n[j].transpose(),D]),f_n[j]]))
    sorted_idx = np.argsort(lr)
    return sorted_idx[:k]